# Boat activity monitoring
In this notebook we will use Descartes Labs `Workflows` to build an interactive map to **monitor river boat traffic on the Yangtze River in China**.  In the map below, you can draw a custom AOI on the Yangtze River (or any water body in the world), and we use synthetic aperture radar (SAR) data from Sentinel-1 to detect the presence of boats within that AOI over the past year.

You can run the following cells using `Shift-Enter`.

## Import packages

In [1]:
# keep logging quiet
import logging
logging.getLogger().setLevel(logging.INFO)
logging.captureWarnings(True)

In [2]:
# import packages
import descarteslabs.workflows as wf
from boat_activity_utils import TimeSeries

## Define the image layers that we will visualize in the map below

#### Synthetic Aperture Radar (SAR) data from Sentinel-1

In [3]:
sar = (
        wf.ImageCollection.from_id('sentinel-1:GRD',
                                   start_datetime='2019-03-01',
                                   end_datetime='2020-04-01')
        .pick_bands(['vv', 'vh'])
      )

# Create base map from the Sentinel-1 SAR
base_sar = sar.tail(2).median(axis='images').mean(axis='bands')
lyr = base_sar.visualize('SAR', scales=[[0.0, 0.3]], colormap='viridis', checkerboard=False)
lyr.visible = False

#### Visble (red green blue) imagery from Sentinel-2

In [4]:
rgb = (
        wf.ImageCollection.from_id('sentinel-2:L1C',
                                   start_datetime='2019-08-01',
                                   end_datetime='2019-12-01')
        .pick_bands(['red', 'green', 'blue'])
        .filter(lambda img: img.properties['cloud_fraction'] < 0.1)
      )

# Create RGB map from Sentinel-2
base_rgb = rgb.min(axis='images') # a minimum composite is good for avoiding haze
base_rgb.visualize('RGB', scales=[[0.0, 0.3], [0.0, 0.3], [0.0, 0.3]], checkerboard=False)

In [5]:
test1 = (
        wf.ImageCollection.from_id('airbus:oneatlas:spot:v2',
                                   start_datetime='2019-08-01',
                                   end_datetime='2019-12-01')
        .pick_bands(['red', 'green', 'blue'])
 #       .filter(lambda img: img.properties['cloud_fraction'] < 0.1)
      )

# Create RGB map from Sentinel-2
test = test1.min(axis='images') # a minimum composite is good for avoiding haze
test.visualize('SPOT', scales=[[0.0, 255], [0.0, 255], [0.0, 255]], checkerboard=False)

## Process the SAR layer to create a layer tracking the presence of boats.

In [6]:
# Create a mask based on thresholding the SAR signal, taking the average over all
# images and thresholding the average
mask = ((sar.mean(axis='bands')>0.02).mean(axis='images')>0.95)

# Threshold the SAR signal to get pixels with high value and apply the mask to only
# include objects over water
thresh = (sar.mean(axis='bands')>0.25).mask(mask)

# Apply a spatial convolution to smooth the detection layer
w = wf.Kernel(dims=(5,5), data=[1.0, 1.0, 1.0, 1.0, 1.0, \
                                1.0, 2.0, 3.0, 2.0, 1.0, \
                                1.0, 3.0, 4.0, 3.0, 1.0, \
                                1.0, 2.0, 3.0, 2.0, 1.0, \
                                1.0, 1.0, 1.0, 1.0, 1.0])
conv = wf.conv2d(thresh.sum(axis='images'), w)

# Display on the map below
conv.visualize('Average Boats', scales=[[0.0, 100.0]], colormap='cividis', checkerboard=False)

## Create the custom widget, imported from `boat_activity_utils.py`, that calculates the historical time series of boat activity within the user-defined Area of Interest (AOI).

In [7]:
# Create time series
boats_timeseries = TimeSeries(wf.map, bin_month=False)

## Define the Area of Interest (AOI)

In [8]:
wf.map.center = 30.5095, 111.4354 # Yangtze River, China, 1000 km from Pacific Ocean.
wf.map.height = 1200
wf.map.zoom = 12
wf.map.map.layout.height="500px"

## Finally, display and interact with the map.
Yon can use the draw tools on the left side of the map to define your AOI for the historical analysis.

In [9]:
wf.map



`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.


### If desired, return the AOI boat counts as a pandas `dataframe`.
You must first draw an AOI in the map above.  Then uncomment and run the line below.

In [10]:
# boats_timeseries.df

In [11]:
#try midland